<a href="https://colab.research.google.com/github/juanxo90/cargamos/blob/main/cargamos_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prueba técnica para la posición de data science en Cargamos



In [1]:
# importamos las librerias
import pandas as pd

In [7]:
pd.set_option("display.max_columns", None)

# cargamos los datos
paq_det = pd.read_csv("https://raw.githubusercontent.com/juanxo90/cargamos/main/data/paquete_detail%20(2).csv")
reg_det = pd.read_csv("https://raw.githubusercontent.com/juanxo90/cargamos/main/data/register_detail%20(2).csv")

In [8]:
paq_det.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14056 entries, 0 to 14055
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id_paquete        14056 non-null  object
 1   cedis             14056 non-null  object
 2   estado            14056 non-null  object
 3   hora_recolectado  13997 non-null  object
 4   hora_creacion     14056 non-null  object
 5   hora_entregado    12502 non-null  object
 6   hora_falla        1651 non-null   object
dtypes: object(7)
memory usage: 768.8+ KB


In [9]:
reg_det.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15573 entries, 0 to 15572
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_paquete          15573 non-null  object
 1   id_monitoreo        15573 non-null  object
 2   arreglo_estados     15573 non-null  object
 3   complemento_estado  15573 non-null  object
 4   fecha_creado        15573 non-null  object
dtypes: object(5)
memory usage: 608.4+ KB


In [10]:
reg_det.arreglo_estados = reg_det.arreglo_estados.str.replace('[', '', regex=True)
reg_det.arreglo_estados = reg_det.arreglo_estados.str.replace(']', '', regex=True)

reg_det.complemento_estado = reg_det.complemento_estado.str.replace('[', '', regex=True)
reg_det.complemento_estado = reg_det.complemento_estado.str.replace(']', '', regex=True)

In [11]:
reg_det

,id_paquete,id_monitoreo,arreglo_estados,complemento_estado,fecha_creado
0,e3868099-1cfc-494d-ae19-09cdf704f735,W2H08FJPELTFWWDK2W43,"DELIVERED,NEXT_IN_ROUTE,OUT_FOR_DELIVERY,NEXT_...","ORIGIN_SCAN,ORIGIN_SCAN",2021-12-10T11:34:47.972000
1,cfba004c-3576-4159-9980-70516e5d35c5,4YNRI0AKHTJCS4DRQFKN,"DELIVERED,DELIVERED,DELIVERED,DELIVERED,NEXT_I...",ORIGIN_SCAN,2021-12-09T20:32:18.163000
2,11946177-b239-48cc-ba33-b135380c6843,HIOSNBGAJXV7TK9Y46MV,"DELIVERED,DELIVERED,DELIVERED,DELIVERED,NEXT_I...","ORIGIN_SCAN,ORIGIN_SCAN",2021-12-09T17:56:20.002000
3,522f7b7f-f91b-49ac-879f-df902514d36d,U5WP205OCXAVNC4NOCND,"DELIVERED,NEXT_IN_ROUTE,NEXT_IN_ROUTE,NEXT_IN_...",ORIGIN_SCAN,2021-12-11T11:52:38.419000
4,98aca3cd-3591-44c3-b494-11aacccb48f0,NA46URBZUSW77ZH1OBNB,"DELIVERED,DELIVERED,DELIVERED,DELIVERED,DELIVE...",ORIGIN_SCAN,2021-12-09T18:39:48.182000
...,...,...,...,...,...
15568,2a17573f-f9b0-465a-a752-f36286598a01,JGZTOUWDQBWKOQEAQKFV,"ARRIVAL_SCAN,ARRIVAL_SCAN,NEXT_IN_ROUTE,OUT_FO...","RETURNED_TO_CLIENT,DELIVERY_ATTEMPTED,ORIGIN_SCAN",2021-12-11T12:00:32.888000
15569,9f676f47-90e9-4f53-ad87-229a3b2f2ad8,L2TGZ3VOQOGGMIHGDTBL,"ARRIVAL_SCAN,ARRIVAL_SCAN,NEXT_IN_ROUTE,OUT_FO...","RETURNED_TO_CLIENT,DELIVERY_ATTEMPTED,ORIGIN_SCAN",2021-12-11T12:51:25.260000
15570,7a81c3e0-d067-4225-a6d6-9aa46a730398,HTDNO9BZZTFMCJYCHE2I,"ARRIVAL_SCAN,ARRIVAL_SCAN,NEXT_IN_ROUTE,OUT_FO...","RETURNED_TO_CLIENT,DELIVERY_ATTEMPTED,ORIGIN_SCAN",2021-12-11T11:04:35.651000
15571,a5db608a-a5ff-46b3-8751-781cb811e6de,EY0HHDDTDG0BUTADFKFZ,"ARRIVAL_SCAN,ARRIVAL_SCAN,NEXT_IN_ROUTE,OUT_FO...","RETURNED_TO_CLIENT,DELIVERY_ATTEMPTED,ORIGIN_SCAN",2021-12-11T11:25:29.200000


In [12]:
def Convert(string):
    li = list(string.split(","))
    return li[0]

reg_det.arreglo_estados = reg_det.arreglo_estados.apply(Convert)
reg_det.complemento_estado = reg_det.complemento_estado.apply(Convert)

In [13]:
reg_det

,id_paquete,id_monitoreo,arreglo_estados,complemento_estado,fecha_creado
0,e3868099-1cfc-494d-ae19-09cdf704f735,W2H08FJPELTFWWDK2W43,DELIVERED,ORIGIN_SCAN,2021-12-10T11:34:47.972000
1,cfba004c-3576-4159-9980-70516e5d35c5,4YNRI0AKHTJCS4DRQFKN,DELIVERED,ORIGIN_SCAN,2021-12-09T20:32:18.163000
2,11946177-b239-48cc-ba33-b135380c6843,HIOSNBGAJXV7TK9Y46MV,DELIVERED,ORIGIN_SCAN,2021-12-09T17:56:20.002000
3,522f7b7f-f91b-49ac-879f-df902514d36d,U5WP205OCXAVNC4NOCND,DELIVERED,ORIGIN_SCAN,2021-12-11T11:52:38.419000
4,98aca3cd-3591-44c3-b494-11aacccb48f0,NA46URBZUSW77ZH1OBNB,DELIVERED,ORIGIN_SCAN,2021-12-09T18:39:48.182000
...,...,...,...,...,...
15568,2a17573f-f9b0-465a-a752-f36286598a01,JGZTOUWDQBWKOQEAQKFV,ARRIVAL_SCAN,RETURNED_TO_CLIENT,2021-12-11T12:00:32.888000
15569,9f676f47-90e9-4f53-ad87-229a3b2f2ad8,L2TGZ3VOQOGGMIHGDTBL,ARRIVAL_SCAN,RETURNED_TO_CLIENT,2021-12-11T12:51:25.260000
15570,7a81c3e0-d067-4225-a6d6-9aa46a730398,HTDNO9BZZTFMCJYCHE2I,ARRIVAL_SCAN,RETURNED_TO_CLIENT,2021-12-11T11:04:35.651000
15571,a5db608a-a5ff-46b3-8751-781cb811e6de,EY0HHDDTDG0BUTADFKFZ,ARRIVAL_SCAN,RETURNED_TO_CLIENT,2021-12-11T11:25:29.200000


In [15]:
df = reg_det.merge(paq_det, how="inner", on=["id_paquete"])

In [21]:
df

,id_paquete,id_monitoreo,arreglo_estados,complemento_estado,fecha_creado,cedis,estado,hora_recolectado,hora_creacion,hora_entregado,hora_falla
0,e3868099-1cfc-494d-ae19-09cdf704f735,W2H08FJPELTFWWDK2W43,DELIVERED,ORIGIN_SCAN,2021-12-10T11:34:47.972000,QUERETARO 002,SUCCESSFUL,2021-12-11T09:55:31.667000,2021-12-10T11:34:35.026000,2021-12-11T10:37:08.450000,NaN
1,cfba004c-3576-4159-9980-70516e5d35c5,4YNRI0AKHTJCS4DRQFKN,DELIVERED,ORIGIN_SCAN,2021-12-09T20:32:18.163000,CUERNAVACA 001,SUCCESSFUL,2021-12-10T12:08:40.646000,2021-12-09T20:32:11.038000,2021-12-10T13:20:11.205000,NaN
2,11946177-b239-48cc-ba33-b135380c6843,HIOSNBGAJXV7TK9Y46MV,DELIVERED,ORIGIN_SCAN,2021-12-09T17:56:20.002000,TULUM 001,SUCCESSFUL,2021-12-10T12:33:10.985000,2021-12-09T17:56:11.983000,2021-12-10T13:32:39.849000,NaN
3,98aca3cd-3591-44c3-b494-11aacccb48f0,NA46URBZUSW77ZH1OBNB,DELIVERED,ORIGIN_SCAN,2021-12-09T18:39:48.182000,TEPOZOTLAN 001,SUCCESSFUL,2021-12-10T08:52:25.562000,2021-12-09T18:39:40.368000,2021-12-10T11:38:50.077000,NaN
4,3be88e84-879f-4bbc-9f3c-0e4a27a9e270,WMHIZY8ZHRJQYU8QSSEK,DELIVERED,ORIGIN_SCAN,2021-12-11T11:14:40.065000,GDL 003,SUCCESSFUL,2021-12-12T09:27:57.979000,2021-12-11T11:14:31.918000,2021-12-12T11:16:22.599000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
13195,7b0539b8-195d-4e83-88f4-0ca261eb0aab,E3WIZ1FR0HSU3FVICV5D,ARRIVAL_SCAN,RETURNED_TO_CLIENT,2021-12-08T11:30:57.868000,TULUM 001,FAILED,2021-12-08T16:09:05.923000,2021-12-08T11:30:50.076000,NaN,2021-12-08T18:59:57.107000
13196,a4585175-a5e8-45f8-818a-9d7756ded415,QMJTXBZ7TBKBA59ISS7I,ARRIVAL_SCAN,RETURNED_TO_CLIENT,2021-12-11T12:19:57.045000,SPAIN 001,FAILED,2021-12-11T16:51:35.023000,2021-12-11T12:19:49.574000,NaN,2021-12-11T18:39:37.616000
13197,9f676f47-90e9-4f53-ad87-229a3b2f2ad8,L2TGZ3VOQOGGMIHGDTBL,ARRIVAL_SCAN,RETURNED_TO_CLIENT,2021-12-11T12:51:25.260000,SPAIN 001,FAILED,2021-12-11T16:51:35.023000,2021-12-11T12:51:16.546000,NaN,2021-12-11T18:38:54.137000
13198,7a81c3e0-d067-4225-a6d6-9aa46a730398,HTDNO9BZZTFMCJYCHE2I,ARRIVAL_SCAN,RETURNED_TO_CLIENT,2021-12-11T11:04:35.651000,CUERNAVACA 001,FAILED,2021-12-11T15:48:02.695000,2021-12-11T11:04:28.256000,NaN,2021-12-11T16:59:12.494000
